Note
1. You run the flask app, check for port and verify_token (current port = 6000 and verify token = 'abc')
2. You run ngrok, you run command ngrok http 6000 to exposed your flask app to port 6000
3. You go to fb graph api => Ứng Dụng => APP: API Testing => Webhooks (left hand side) => Edit subscription of PAGE!!!, not user
4. On ngrok app, copy the forwarding address, then Pass it to the callback url, along with the verify token = 'abc'
5. Test the webhooks to see returned data (User => birthday)

In [13]:
import urllib.parse
import requests
import json

In [14]:
import json

# Đường dẫn tới tệp JSON
config_file = 'config.json'

# Đọc tệp JSON
with open(config_file, 'r') as file:
    config = json.load(file)

# Lấy thông tin từ tệp JSON
app_id = config.get('app_id')
app_secret = config.get('app_secret')

In [3]:
# # Your Facebook App credentials

# redirect_uri = 'https://www.loreal.com/vi-vn/vietnam/'
# code = 'AQAcsfoSAOw18o5xCr5h1hnzwFv3cumzWeutZWyvcKA3Ep9ilCL7BzaP4tWLvoWlXME_QMsiv6CqepBpOHPfsYFoxoW16_Ef_psd7xiwMCr8kMJcPCwqKHRS7KsYzlqoTlFFJ-QbI2AEGzcacb0_1ly8hgrfeRQPY6bRLjeVrJgz28IMZkgDLbzxvSC0vO2SsRzKd_zTyp_EH33P-WayJvinOOnNpQ3IFz7Pwun0TRAhV938igQuWTYEj6Ul2HonHABPwKv9aOw2ZP4LSTl417uC7cLmz1fmShlfsiQje08HwIJ7bTdKUcGojiT_zN3G-fyZ1IfgkzZm0p3zk7_ymMqwTzHBPiCGOIg4DzwI_naSyMXaJnAUSnpcKkIpu-XytVKDH9HM1NCpnGgRLBwuZrFkFBBWbzfhjarVMGW1_les2Q#_=_'

# # Construct the URL to exchange the code for an access token
# token_url = (
#     f"https://graph.facebook.com/v10.0/oauth/access_token?"
#     f"client_id={app_id}&"
#     f"redirect_uri={redirect_uri}&"
#     f"client_secret={app_secret}&"
#     f"code={code}"
# )

# # Make the request to get the access token
# response = requests.get(token_url)
# access_token_info = response.json()

In [15]:
# Facebook bot/access_token_info.json

with open('C:/Users/admin/Facebook bot/access_token_info.json', 'r') as file:
    access_token_info = json.load(file)
    
access_token = access_token_info['access_token']

In [16]:
long_lived_token = access_token

def extend_long_lived_token(long_lived_token, app_id, app_secret):
    url = 'https://graph.facebook.com/v13.0/oauth/access_token'
    params = {
        'grant_type': 'fb_exchange_token',
        'client_id': app_id,
        'client_secret': app_secret,
        'fb_exchange_token': long_lived_token
    }
    response = requests.get(url, params=params)
    return response.json()

long_lived_token_info = extend_long_lived_token(long_lived_token, app_id, app_secret)
long_lived_token_info

{'access_token': 'EAAHvZCV461MgBOywEWdKobGZCRr9TC1otW7ZC37VSZBLdE0oIf8n2vYXU1moirg0jWpfl48AuYT8gxslYxWxiMsHnjEpZAYagwqYRhGw7d6kWEg4BOWwtalEc1SZBvE1sALP78D8ArjL3QIeeEm7v3tE5ZAOCxHnXbIqQi6ahZA2WV7vOWt5LqsxqdZCv',
 'token_type': 'bearer'}

In [17]:
with open('access_token_info.json', 'w') as file:
    json.dump(long_lived_token_info, file, indent=4)

## 2. Get page authorization

In [6]:
# Construct the URL to get the list of pages
url = f'https://graph.facebook.com/v10.0/me/accounts?access_token={access_token}'

# Make the request
response = requests.get(url)
data = response.json()['data']
filtered_data = [item for item in data if item['name'] == 'Netflix otp'][0]
page_access_token = filtered_data['access_token']
page_id = filtered_data['id']
filtered_data

{'access_token': 'EAAHvZCV461MgBO49KCvBZCZCDDpVU5nQ2sAzfRbOAMJMNheZCutAUTA9KZA4wmZCAVrW41vy59nt761HgpiIawf18JLFKuOudJztqdj1bcloSOZAiqRDZAmOc0XhGLRcZB6i6hTX1cTFxIwASnOQfzoT8CGLsvAFzkC5iI7Ei2isT8osDht7M5WZBUrh4gLmCpIqxT',
 'category': 'Chương trình TV',
 'category_list': [{'id': '1400', 'name': 'Chương trình TV'}],
 'name': 'Netflix otp',
 'id': '345243362013764',
 'tasks': ['ADVERTISE',
  'ANALYZE',
  'CREATE_CONTENT',
  'MESSAGING',
  'MODERATE',
  'MANAGE']}

## 3. Get all message

In [8]:
# def get_messages(page_id, access_token):
#     url = f'https://graph.facebook.com/v13.0/{page_id}/conversations'
#     params = {
#         'access_token': access_token
#     }
#     response = requests.get(url, params=params)

# def get_conversation_threads(conversation_id, access_token):
#     url = f'https://graph.facebook.com/v20.0/{conversation_id}'
#     params = {
#         'fields': 'participants, messages',  # Request specific fields
#         'access_token': access_token
#     }
#     response = requests.get(url, params=params)
    
# def get_message_details(message_id, access_token):
#     url = f'https://graph.facebook.com/v13.0/{message_id}'
#     params = {
#         'fields': 'message,from,to,created_time',  # Request specific fields
#         'access_token': access_token
#     }
#     response = requests.get(url, params=params)

In [7]:
import requests

def get_all_messages(page_id, access_token):
    def get_messages(page_id, access_token):
        url = f'https://graph.facebook.com/v13.0/{page_id}/conversations'
        params = {
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        return response.json() if response.status_code == 200 else response.json()

    def get_conversation_threads(conversation_id, access_token):
        url = f'https://graph.facebook.com/v20.0/{conversation_id}'
        params = {
            'fields': 'participants,messages{message,from,to,created_time}',  # Request specific fields
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        return response.json() if response.status_code == 200 else response.json()

    # Fetch conversations
    conversations = get_messages(page_id, access_token)
    all_messages = []

    # Process each conversation
    for conversation in conversations.get('data', []):
        conversation_id = conversation['id']
        threads = get_conversation_threads(conversation_id, access_token)
        
        # Extract participants
        participants = {p['id']: p['name'] for p in threads.get('participants', {}).get('data', [])}

        # Extract messages
        messages = threads.get('messages', {}).get('data', [])
        for message in messages:
            message_content = message.get('message', '')
            created_time = message.get('created_time', '')
            from_user_id = message.get('from', {}).get('id', '')
            from_user_name = participants.get(from_user_id, '')

            # Ensure from_user_id and from_user_name are defined
            if from_user_id and from_user_name:
                # Find 'to' details
                to_users = threads.get('participants', {}).get('data', [])
                for recipient in to_users:
                    if recipient['id'] != from_user_id:  # Exclude sender
                        all_messages.append({
                            'message': message_content,
                            'sender_id': from_user_id,
                            'sender_name': from_user_name,
                            'recipient': recipient['name'],
                            'recipient_id': recipient['id'],
                            'created_time': created_time
                        })
    
    return all_messages

# Example usage
messages = get_all_messages(page_id, page_access_token)
messages[0:5]

[{'message': 'abc',
  'sender_id': '8176163065778443',
  'sender_name': 'Bình Như Gia Lê',
  'recipient': 'Netflix otp',
  'recipient_id': '345243362013764',
  'created_time': '2024-10-05T11:45:26+0000'},
 {'message': 'xy',
  'sender_id': '8176163065778443',
  'sender_name': 'Bình Như Gia Lê',
  'recipient': 'Netflix otp',
  'recipient_id': '345243362013764',
  'created_time': '2024-09-30T16:29:19+0000'},
 {'message': 'sup',
  'sender_id': '8176163065778443',
  'sender_name': 'Bình Như Gia Lê',
  'recipient': 'Netflix otp',
  'recipient_id': '345243362013764',
  'created_time': '2024-09-30T02:41:00+0000'},
 {'message': 'abc',
  'sender_id': '8176163065778443',
  'sender_name': 'Bình Như Gia Lê',
  'recipient': 'Netflix otp',
  'recipient_id': '345243362013764',
  'created_time': '2024-09-30T02:19:59+0000'},
 {'message': '789',
  'sender_id': '8176163065778443',
  'sender_name': 'Bình Như Gia Lê',
  'recipient': 'Netflix otp',
  'recipient_id': '345243362013764',
  'created_time': '2024

## 4. send message

In [19]:
def send_message(page_access_token, recipient_id, message_text):
    url = 'https://graph.facebook.com/v13.0/me/messages'
    params = {
        'access_token': page_access_token
    }
    payload = {
        'recipient': {
            'id': recipient_id
        },
        'message': {
            'text': message_text
        }
    }
    response = requests.post(url, params=params, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        return response.json()

# Example usage
# result = send_message(page_access_token, recipient_id, message_text)
# print(result)

## 5. run while loop to check for msg

In [20]:
import time
import random

# Define the sender ID to exclude
excluded_sender_id = '345243362013764'
# start with 8
recipient_id = '8176163065778443'
# message_text = 'Wat suppp!'
message_lst = [
    'hello', 'hi', 'xin chao'
]

# Initial fetch of messages
current_messages = get_all_messages(page_id, page_access_token)
previous_messages = current_messages

while True:
    # Fetch current messages
    current_messages = get_all_messages(page_id, page_access_token)
    
    # Find new messages excluding those from the specific sender
    new_messages = [msg for msg in current_messages if msg not in previous_messages and msg['sender_id'] != excluded_sender_id]
    
    if new_messages:
        # Choose a random message from the list
        random_message = random.choice(message_lst)
        
        # Send the random message
        send_message(page_access_token, recipient_id, random_message)
        
        print(new_messages[0])
    
    # Update the list of previously fetched messages
    previous_messages = current_messages
    
    # Wait for 2 seconds before checking again
    time.sleep(0.5)

{'message': 'hello', 'sender_id': '8176163065778443', 'sender_name': 'Bình Như Gia Lê', 'recipient': 'Netflix otp', 'recipient_id': '345243362013764', 'created_time': '2024-07-28T10:21:36+0000'}
{'message': 'binh dep trai', 'sender_id': '8176163065778443', 'sender_name': 'Bình Như Gia Lê', 'recipient': 'Netflix otp', 'recipient_id': '345243362013764', 'created_time': '2024-07-28T10:21:59+0000'}


KeyboardInterrupt: 

## 6. Trigger a gpt api call when the message_log file is updated